In [ ]:
import glob
import os
import sys

In [ ]:
try:
    sys.path.append(glob.glob('/opt/carla-simulator/PythonAPI/carla/dist/carla-*3.7-%s.egg' % (
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

In [3]:
import carla

In [4]:
import random
import time
import numpy as np
import cv2

In [5]:
IM_WIDTH = 640
IM_HEIGHT = 480
actor_list = []

In [6]:
def process_img(image):
    i = np.array(image.raw_data)
    i2 = i.reshape((IM_HEIGHT, IM_WIDTH, 4))
    i3 = i2[:, :, :3]
    cv2.imshow("image", i3)
    cv2.waitKey(1)
    return i3/255.0

In [7]:
try:
    client = carla.Client("localhost", 2000)
    client.set_timeout(2.0)
    world = client.get_world()
    blueprint_library = world.get_blueprint_library()
    
    bp = blueprint_library.filter("model3")[0]
    print(bp)
    
    spawn_point = random.choice(world.get_map().get_spawn_points())
    
    vehicle = world.spawn_actor(bp, spawn_point)
#     vehicle.set_autopilot(True)
    vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0.0))
    actor_list.append(vehicle)
    
    cam_bp = blueprint_library.find("sensor.camera.rgb")
    cam_bp.set_attribute("image_size_x", f"{IM_WIDTH}")
    cam_bp.set_attribute("image_size_y", f"{IM_HEIGHT}")
    cam_bp.set_attribute("fov", "110")
    
    spawn_point = carla.Transform(carla.Location(x=2.5, z=0.7))
    
    sensor = world.spawn_actor(cam_bp, spawn_point, attach_to=vehicle)
    actor_list.append(sensor)
    sensor.listen(lambda data: process_img(data))
    
    time.sleep(5)
    
finally:
    for actor in actor_list:
        actor.destroy()
    print("All cleaned up!")
    
    cv2.destroyAllWindows() 


ActorBlueprint(id=vehicle.tesla.model3,tags=[model3, vehicle, tesla])
All cleaned up!
